In [1]:
# Example of how I calculated the expected values of a set of variables (e.g. silver mints)
# over time. Values are calculated first, regardless of start and end dates, but then we only
# aggregate values from 0-450 AD in this specific use case. No need for another 600 columns to
# look at BC in depth (as was the case for brooches).
def silver_mints_expected_values():
    # open a CSV for the output
    outfile = open('silver_mints_EVs.csv', 'w')
    writer=csv.writer(outfile)
    
    # make a title row with the years 0-450 AD
    title_row = []
    for i in range(0,451):
        title_row.append(i)
    writer.writerow(title_row)

    # Make a new EV tracker for each unique silver mint.
    # The EV tracker is a list; but, if we were starting from any year other than zero,
    # we could not simply use list indexes to reference years. A dictionary can be used
    # in that scenario.
    silver_mints = {}
    for row in SILVER_COINS:
        r = str(row['mintName'])
        if r in silver_mints.keys():
            pass
        else:
            silver_mints[r]=[]
            for i in range(0, 451):
                silver_mints[r].append(0)
            
    # iterate through your dataset (in this case a subset of all silver coins)
    for row in SILVER_COINS:
        # establish variables of interest
        mint = str(row['mintName'])
        n1 = row['numdate1']
        n2 = row['numdate2']
        
        # If you are 100% sure that there are errors, feel free to address. E.g. ...
        if row['primaryRuler']=="Salonina (sole reign of Gallienus)":
            if (n1 == 253.0) and (n2 == 260.0):
                n1 = 260.0
                n2 = 268.0
        # ^ This is a clear error of ambiguation in the PAS dataset. Uploaders sometimes gave the dates for
        # "Salonina (joint reign of Gallienus)"" when they had a coin from Sole Reign (which came after).
                
        # The object's EV in any given year is 1/(the size of the total date range)
        val = 1/(abs(n2-n1)+1)
        
        # In this case we are adjusting coins by their weights. As long as a value has been uploaded,
        # we therefore make the yearly EV (UNIT_MASS) the weight * val (remember, val is less than one)
        weight = row['weight']
        if (not np.isnan(weight)):
            UNIT_MASS = weight*val

            # We indent the rest of the if-statements because we only want coins that have uploaded weights.
            # If a start and end date have both been uploaded, proceed...
            if (not np.isnan(n1)) and (not np.isnan(n2)):
                # Though we have calculated the yearly values for the full range of the object, sometimes we only
                # want to aggregate the "yearly buckets" in a specific range. To do so, simply set the start and end
                # dates to 0/450 if the object has dates <0 or >450. IMPORTANT: The yearly values MUST be calculated 
                # first, otherwise you may accidentally change the range of the object itself, rather than the range
                # of aggregated "yearly buckets."
                if n1 < 0:
                    start = 0
                else:
                    start = int(n1)
                if n2 > 450:
                    end = 450
                else:
                    end = int(n2)
                    
                # Get the current expected value tracker for the coin's silver mint.
                temp = silver_mints[mint]
            
                # Update the tracker with the EVs of the coin
                for i in range(start, (end+1)):
                    temp[i]+=UNIT_MASS
                silver_mints[mint]=temp
                
    # After iterating through all the rows, output the results to the CSV.
    for item in silver_mints.keys():
        data = silver_mints[item]
        data.append(item)
        writer.writerow(data)
    outfile.close()
